In [ ]:
import boto3

ddb = boto3.client('dynamodb')

In [ ]:
response = ddb.create_table(
    TableName='EventStore',
     AttributeDefinitions=[
        {
            'AttributeName': 'AggregateId',
            'AttributeType': 'S'
        },
        {
            'AttributeName': 'Version',
            'AttributeType': 'N'
        }
    ],
    KeySchema=[
        {
            'AttributeName': 'AggregateId',
            'KeyType': 'HASH'
        },
        {
            'AttributeName': 'Version',
            'KeyType': 'RANGE'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 10,
        'WriteCapacityUnits': 5
    }
)

print response

In [ ]:
class Event:
    kind = 'esevent'
    
    def __init__(self, id, version, typecode, payload):
        self.id = id
        self.version = version
        self.typecode = typecode
        self.payload = payload
        
    def __str__(self):
        return 'Event {} - {}'.format(self.id. self.version)

In [ ]:
addUser = {}
addUser['Name'] = 'doug'
addUser['Email'] = 'doug@dev.null'
type(addUser)


In [ ]:
class UserAdded(Event):
    kind = 'useradded'

class EmailUpdated(Event):
    kind = 'emailupdated'
    
def print_event_kind(event):
    print type(event)
    print event.kind


In [ ]:
e1 = UserAdded('ag1',1, 'adduser', addUser)
print_event_kind(e1)

In [ ]:
print e1.id
print type(e1)

In [ ]:
updateEmail = {}
updateEmail['Email'] = "reallydoug@somewhere.com"
e2 = EmailUpdated('ag1',2, 'updateemail', updateEmail)

In [ ]:
from boto3.dynamodb.types import TypeSerializer


ts = TypeSerializer()
addUserMap = ts.serialize(addUser)
print addUserMap

In [ ]:
response = ddb.batch_write_item(
    RequestItems={
        'EventStore':[
            {
                'PutRequest':{
                    'Item':{
                        'AggregateId':{'S': e1.id},
                        'Version':{'N':str(e1.version)},
                        'Payload':ts.serialize(e1.payload),
                        'Typecode':{'S':e1.typecode}
                    }
                }
            }
        ]
    }
)

print response

In [ ]:

response = ddb.batch_write_item(
    RequestItems={
        'EventStore':[
            {
                'PutRequest':{
                    'Item':{
                        'AggregateId':{'S': e2.id},
                        'Version':{'N':str(e2.version)},
                        'Payload':ts.serialize(e2.payload),
                        'Typecode':{'S':e2.typecode}
                    }
                }
            }
        ]
    }
)

print response

In [ ]:
from boto3.dynamodb.conditions import Key

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('EventStore')
response = table.query(
    KeyConditionExpression=Key('AggregateId').eq(e1.id)
)

print response['Items']

payload1 = response['Items'][0]['Payload']
print payload1
print type(payload1)

In [ ]:
response = table.query(
    KeyConditionExpression=Key('AggregateId').eq(e1.id),
    ScanIndexForward=False
)

print response

In [ ]:
import uuid

class Aggregate(object):
    def __init__(self):
        self.AggregateID = uuid.uuid4()
        self.Events = []
        self.Version = 0
        
    def apply_event(self,event):
        print 'implement this'

    def route_event(event):
        print 'implement this'

    def store(self):
        print 'implement this'

In [ ]:
# The class representing the event source entity
class TestAgg(Aggregate):
    def __init__(self, foo = '', bar='', baz='',event_history=None):
        Aggregate.__init__(self)
        self.Foo = ''
        self.Bar = ''
        self.Baz = ''
                
        if event_history != None: 
            self.apply_history(event_history)
        else:
            self.Version = 1
            self.__init_via_args(foo, bar, baz)
    
    def __init_via_args(self, foo, bar, baz):
        aggCreated = TestAggCreated()
        aggCreated.AggregateID = self.AggregateID
        aggCreated.Foo = foo
        aggCreated.Bar = bar
        aggCreated.Baz = bar

        self.apply_event(aggCreated)
        
    def apply_history(self,event_history):
        print 'apply_history called with {}'.format(event_history)
        for e in event_history:
            self.Version += 1
            self.route_event(e)
            
    def route_event(self, event):
        event.version = self.Version
        print type(event)
        if '.TestAggCreated' in str(type(event)):
            self.handle_create(event)
        else:
            print 'unknown event'
            
    def apply_event(self, event):
        self.route_event(event)
        self.Events.append(event)
        
    def handle_create(self, event):
        print 'handle_create'
        self.AggregateID = event.AggregateID
        self.Foo = event.Foo
        self.Bar = event.Bar
        self.Baz = event.Baz
            
        
    def Dump(self):
        print 'agg id: {}'.format(self.AggregateID)
        print 'events: {}'.format(self.Events)
        print 'version: {}'.format(self.Version)
        print 'foo: {}'.format(self.Foo)
        print 'bar: {}'.format(self.Bar)
        print 'baz: {}'.format(self.Baz)

In [ ]:
# Event classes - forgoing the use of protobufs for now to keep things simple
class TestAggCreated(object):
    def __init__(self):
        self.AggregateID = ''
        self.Foo = ''
        self.Bar = ''
        self.Baz = ''

In [ ]:
class BarUpdated(object):
    def __init__(self,bar):
        self.NewBar = bar

In [ ]:
tag = TestAggCreated()
print tag.AggregateID
tag.AggregateID = '123'
tag.Foo = 'This is a foo'
tag.Baz = 'This is some baz'
print tag.AggregateID
print tag.Foo

In [ ]:
myTestAgg = TestAgg('my foo', 'my bar', 'your baz')

In [ ]:
myTestAgg.Dump()

In [ ]:
print type(myTestAgg)

In [ ]:
ag = Aggregate()
dir(ag)
dir(myTestAgg)

In [ ]:
events = [tag]
ta2 = TestAgg(event_history=events)
ta2.Dump()

In [ ]:
print isinstance(ta2,TestAgg)
print isinstance(ta2, Aggregate)
print ta2.store()

In [ ]:
class ConcurrencyException(Exception):
    def __init__(self, value):
        self.value = value
        
    def __str__(self):
        return repr(self.value)

In [ ]:
try:
    raise ConcurrencyException('Last version was 9')
except ConcurrencyException as foo:
    print foo

In [ ]:
# Storing stuff in a map
eventsPerAggregate = {}
eventsPerAggregate[myTestAgg.AggregateID] = myTestAgg.Events


In [ ]:
ta3 = TestAgg(event_history=eventsPerAggregate[myTestAgg.AggregateID])
ta3.Dump()

In [ ]:
# Next: structure for event storage - current version and events so we can detect concurrency issues